# Numerical Methods 1
### [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman), [Matthew Piggott](http://www.imperial.ac.uk/people/m.d.piggott)

# Lecture 3: Numerical Linear Algebra II

## Learning objectives:

* More on direct methods: LU decomposition
* Doolittle's algorithm
* Properties of lower-triangular matrices
* Partial pivoting

## Introduction

Last week we developed and implemented the Gaussian elimination method to solve the linear matrix system ($A\pmb{x}=\pmb{b}$). 

This week we will consider a closely related solution method: *LU decomposition* or *LU factorisation*.

Both are examples of *direct* solution methods - next week we will consider the alternate approach to solve linear systems, namely iterative or indirect methods.

## LU decomposition - theory

Last week we implemented Gaussian elimination to solve the matrix system with *one* RHS vector $\pmb{b}$.  

We often have to deal with problems where we have multiple RHS vectors, all with the same matrix $A$.  

We could call the same code multiple times to solve all of these corresponding linear systems, but note that as the elimination algorithm is actually performing operations based upon (the same) $A$ each time, we would actually be wasting time repeating exactly the same operations - this is therefore clearly not an efficient solution to this problem.

We could easily generalise our Gaussian elimination/back subsitution algorithms to include multiple RHS column vectors in the augmented system, perform the same sequence of row operations (but now only once) to transform the matrix to upper-triangular form, and then perform back substitution on each of the transformed RHS vectors from the augmented system - cf. the use of Gaussian elimination to compute the inverse to a matrix by placing the identity on the right of the augmented system.

However, it is often the case that each RHS vector depends on the solutions to the matrix systems obtained from some or all of the earlier RHS vectors, and so this generalisation would not work in this case. Note that an example of this you will see in NM2 is where you are time-stepping the solution to a differential equation, and the RHS vector depends on the solution at the previous time level.

To deal with this situation efficiently we *decompose* or *factorise* the matrix $A$ in such a way that it is cheap to compute a new solution vector $\pmb{x}$ for any given RHS vector $\pmb{b}$.  This decompisition involves a lower- and an upper-triangular matrix, hence the name LU decomposition. These matrices essentially *encode* the steps conducted in Gaussian elimination, so we don't have to explicilty conduct all of the operations again and again.

Mathematically, let's assume that we have already found/constructed a lower-triangular matrix ($L$ - where all entries above the diagonal are zero) and an upper-triangular matrix ($U$ - where all entries above the diagonal are zero) such that we can write

$$ A = LU $$

In this case the matrix system we need to solve for $\pmb{x}$ becomes

$$ A\pmb{x} = \pmb{b} \iff (LU)\pmb{x} = L(U\pmb{x}) = \pmb{b} $$

Notice that the matrix-vector product $U\pmb{x}$ is itself a vector, let's call it $\pmb{c}$ for the time-being (i.e. 
$\pmb{c}=U\pmb{x}$).

The above system then reads 

$$ L\pmb{c} = \pmb{b} $$

where $L$ is a matrix and $\pmb{c}$ is an unknown.  

As $L$ is in lower-triangular form we can use forward substitution (generalising the back subsitution algorithm/code we developed last week) to very easily find $\pmb{c}$ in relatively few operations (we don't need to call the entire Gaussian elimination algorithm).

Once we know $\pmb{c}$ we then solve the second linear system 

$$ U\pmb{x} = \pmb{c} $$

where now we can use the fact that $U$ is upper-triangular to use our back substitution algorithm again very efficiently to give the solution $\pmb{x}$ we require.

So for a given $\pmb{b}$ we can find the corresponding $\pmb{x}$ very efficiently, we can therefore do this repeatedly as each new $\pmb{b}$ is given to us.

Our challenge is therefore to find the matrices $L$ and $U$ allowing us to perform the decomposition $A=LU$.


## LU decomposition - algorithm

Recall the comment above on the $L$ and $U$ matrices encoding the steps taken in Gaussian elimination.  Let's see how this works through the development of the so-called Doolittle algorithm.

Let's consider an example matrix:

$$
  A=\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
{\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
{\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
{\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\
    \end{bmatrix}
$$

the first step of Gaussian elimination is to set the
sub-diagonal elements in the first column to zero by subtracting multiples of
the first row from each of the subsequent rows. 

For this example, using the symbolic notiation from last week
this requires the row operations

\begin{align}
Eq. (2) &\leftarrow Eq. (2) - 1\times Eq. (1)\\
Eq. (3) &\leftarrow Eq. (3) - 4\times Eq. (1)\\
Eq. (4) &\leftarrow Eq. (4) - 5\times Eq. (1)\\
\end{align}

or mathematically, and for each element of the matrix (remembering that we are adding rows together - while one of
the entries of a row will end up being zero, this also has the consequence of updating the rest of the values in that row, hence the iteration over $j$ below):

\begin{align}
A_{2j} &\leftarrow A_{2j} - \frac{A_{21}}{A_{11}} A_{1j} = A_{2j} - \frac{5}{5} \times A_{1j}, \quad j=1,2,3,4\\
A_{3j} &\leftarrow A_{3j} - \frac{A_{31}}{A_{11}} A_{1j} = A_{3j} - \frac{20}{5} \times A_{1j}, \quad j=1,2,3,4\\
A_{4j} &\leftarrow A_{4j} - \frac{A_{41}}{A_{11}} A_{1j} = A_{4j} - \frac{25}{5} \times A_{1j}, \quad j=1,2,3,4\\
\end{align}

Notice that we can also write these exact operations on elements in terms of multiplication by a carefully chosen lower-triangular matrix where the non-zero's below the diagonal restricted to a single column, e.g. for the example above

$$
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{Orange}{-1}} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{Orange}{-4}} & {\color{black}0} & {\color{black}1} & {\color{black}0}\\
    {\color{Orange}{-5}} & {\color{black}0} & {\color{black}0} & {\color{black}1}\\   
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
    {\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
    {\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\   
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}{0}} & {\color{blue}{7}} & {\color{blue}{2}} & {\color{blue}{1}}\\
    {\color{blue}{0}} & {\color{blue}{49}} & {\color{blue}{21}} & {\color{blue}{12}}\\
    {\color{blue}{0}} & {\color{blue}{56}} & {\color{blue}{30}} & {\color{blue}{22}}\\    
  \end{bmatrix}
$$

The lower-triangular matrix (let's call this one $L_0$) is thus encoding the first step in Gaussian elimination.

The next step involves taking the second row of the updated matrix as the new pivot (we will ignore partial pivoting for simplicity), and subtracting multiples of this row from those below in order to set the zeros below the diagonal in the second column to zero. 

This can be achieved here with the multiplication by the following lower-triangular matrix (call this one $L_1$)

\begin{equation*}
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{Orange}{-7}} & {\color{black}1} & {\color{black}0}\\
    {\color{black}0} & {\color{Orange}{-8}} & {\color{black}0} & {\color{black}1}\\
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}49} & {\color{black}21} & {\color{black}12}\\
    {\color{black}0} & {\color{black}56} & {\color{black}30} & {\color{black}22}\\
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{blue}{0}} & {\color{blue}{7}} & {\color{blue}{5}}\\
    {\color{black}0} & {\color{blue}{0}} & {\color{blue}{14}} & {\color{blue}{14}}\\
  \end{bmatrix}
\end{equation*}


and finally for this example

\begin{equation*}
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}0} & {\color{black}0} & {\color{black}1} & {\color{black}0}\\
    {\color{black}0} & {\color{black}0} & {\color{Orange}{-2}} & {\color{black}{1}}\\
  \end{bmatrix}\qquad\times\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}0} & {\color{black}7} & {\color{black}5}\\
    {\color{black}0} & {\color{black}0} & {\color{black}14} & {\color{black}14}\\
  \end{bmatrix}\qquad=\qquad\begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}0} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{black}0} & {\color{black}0} & {\color{black}7} & {\color{black}5}\\
    {\color{black}0} & {\color{black}0} & {\color{blue}{0}} & {\color{blue}{4}}\\
  \end{bmatrix}
\end{equation*}

where this lower triangular matrix we'll call $L_2$, and the RHS matrix is now in upper-triangular form as we expect from Gaussian elimination (call this $U$).

In summary, the above operations can be written as 

$$ L_2(L_1(L_0A)) = U $$

(where $A$ here is the original matrix).

Note that these lower-triangular matrices are examples pf what is known as an *atomic* lower-triangular matrix is a special form of unitriangular matrix - the diagonals are unity, and the off-diagonal entries are all zero apart from in a single column. The inverse of such a matrix is the original with the sign of those off-diagnonals changed:

$$
\left[
  \begin{array}{rrrrrrrrr}
    1      & 0      & \cdots      &        &              &        &         &   & 0 \\
    0      & 1      & 0           & \cdots &              &        &         &   & 0 \\
    0      & \ddots & \ddots      & \ddots &              &        &         &   &  \vdots \\
    \vdots & \ddots & \ddots      & \ddots &              &        &         &   &  \\
           &        &             &   0    &   1          &  0     &         & &  &  \\           
           &        &             &   0    &   l_{i+1,i}  &  1     &  \ddots &   &  &  \\  
           &        &             &   0    &   l_{i+2,i}  &  0     &  \ddots &   & &  \\  
           &        &             & \vdots &   \vdots     & \vdots &  \ddots &   & 0 &  \\               
    0      & \cdots &             & 0      &   l_{n,i}    & 0      &  \cdots & 0 & 1 &  \\    
\end{array}
\right]^{-1}
=
\left[
  \begin{array}{rrrrrrrrr}
    1      & 0      & \cdots      &        &              &        &         &   & 0 \\
    0      & 1      & 0           & \cdots &              &        &         &   & 0 \\
    0      & \ddots & \ddots      & \ddots &              &        &         &   &  \vdots \\
    \vdots & \ddots & \ddots      & \ddots &              &        &         &   &  \\
           &        &             &   0    &   1          &  0     &         & &  &  \\           
           &        &             &   0    &   -l_{i+1,i}  &  1     &  \ddots &   &  &  \\  
           &        &             &   0    &   -l_{i+2,i}  &  0     &  \ddots &   & &  \\  
           &        &             & \vdots &   \vdots     & \vdots &  \ddots &   & 0 &  \\               
    0      & \cdots &             & 0      &   -l_{n,i}    & 0      &  \cdots & 0 & 1 &  \\    
\end{array}
\right]
$$


### <span style="color:blue">Exercise 3.1: lower-triangular matrices</span>

Convince yourselves of the following facts:

* The multiplication of arbitrary lower-triangular square matrices is also lower-triangular.

* $L_2(L_1(L_0A)) = U \implies A = L_0^{-1}(L_1^{-1}(L_2^{-1}U))$

* and hence that $A=LU$ where $U$ is the upper-triangular matrix found at the end of Guassian elimination, and where $L$ is the 
following  matrix
$$ L = L_0^{-1}L_1^{-1}L_2^{-1} $$

* Finally, compute this product of these lower-triangular matrices to show that 
$$L = 
  \begin{bmatrix}
    {\color{black}1} & {\color{black}0} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{1}} & {\color{black}1} & {\color{black}0} & {\color{black}0}\\
    {\color{black}{4}} & {\color{black}7} & {\color{black}1} & {\color{black}0}\\
    {\color{black}{5}} & {\color{black}8} & {\color{black}2} & {\color{black}1}\\   
  \end{bmatrix}
$$
i.e. that the multiplication of these individual atomic matrices (importantly in this order) simply merges the entries from the non-zero columns of each atomic matrix, and hence is both lower-triangular, as well as trivial to compute.

In [1]:
import numpy as np
from scipy import linalg
import pprint

# Ex. 3.1 Gaussian elimination (upper-triangular) algorithm from previous lecture
def gauss_elim(A,b):
    # Combine A and b into one array - this is a slightly cleaner way of doing this compared to 
    # carrying around both A and b separately as we had in the implementation last week
    C = np.column_stack((A,b))
    for k in range(np.size(b)-1):
        for j in range(k+1,np.size(b)):
            # Now that they are all one array, the rows can be operated on
            # and we don't bother computing s separately
            C[j,:] -= C[k,:]*C[j,k]/C[k,k]
            
    # Slice array back into A,b  form
    U = C[:,:-1]                                                                                       
    c = C[:,-1]
    return U, c


# Ex. 3.2 A function to perform LU decomposition
def LU_dec(A):
    N  = np.size(A[0,:])
    L  = np.identity(N)
    # Cycle through each row in the Matrix. Use the index 'k'.                                                                             
    for k in range(N):
        # Cycle through each row in the Matrix. Use the index 'k'.
        # From the next row down cycle through each subsequent row. Use the index 'j', j>k.
        for j in range(k+1,N):                                                          
            # The Lower matrix value at position [j,k] 
            # (NB: jth row, kth column) = the coefficient in the gaussian elim
            L[j,k]  = A[j,k]/A[k,k]
            # subtract the pivot row (times the coefficient) from the current row
            A[j,:] -= A[k,:]*L[j,k]
            
    # This step is not needed but explicitly shows A is the Upper Matrix
    U = A                                                                                                   
    return L, U

# Part of Ex. 3.3. This function is not necessary but makes the LU_dec_pp function less cluttered
# Making general operations into functions is good way of improving readability and reducing your workload later!
def swap_rows(A,j,k):
    B = np.copy(A[j,:])
    C = np.copy(A[k,:])
                                                                                                                        # second label, BUT NOT CREATING A 2ND ARRAY. Try it for yourself if you want.
    A[k,:] = B
    A[j,:] = C
    return A

# Ex. 3.3 A function to perform LU decomposition with partial pivoting
def LU_dec_pp(A):
    N  = np.size(A[0,:])
    P_ = np.identity(N)
    L  = np.identity(N)
    for k in range(N):
        j = np.argmax(abs(A[k:,k]))                                                                                                                        # Find the index of the largest ABSOLUTE value. np.argmax will return the index of the largest element in an array
        j+= k                                                                                                           # A[1+2,2] = A[3,2] = 3!
        
        A  = swap_rows(A,j,k)
        P_ = swap_rows(P_,j,k)
        for j in range(k+1,N):
            A[j,k:] -= A[k,k:]*A[j,k]/A[k,k]                                
    U = A   
    return P_.transpose(), L, U
                                                                                                                        # Hence, P = P_^-1! P is the inverse of our array P_ ! But P_ is a rearranged identity matrix, so it will have
                                                                                                                        # a determinant of 1 which means we just need to transpose it to get the inverse! You can try P.P_ yourself


In [2]:
A=np.array([[25., 91. ,55., 67.],
            [ 5., 7.,   5.,  9.],
            [20., 77., 41., 48.],
            [ 5., 14.,  7., 10.]])
B=np.array([[ 5., 7.,   5.,  9.],
            [ 5., 14.,  7., 10.],
            [20., 77., 41., 48.],
            [25., 91. ,55., 67.]])

# We don't need b for this exercise, but the function requires it so 
# simply use an array of the correct shape with zero's.
b = np.zeros((4))                                                                                                       # zeros.
C = np.copy(A)
L1,U1 = LU_dec(C)

C = np.copy(B)
P,L,U = LU_dec_pp(C)

C = np.copy(B)
U2,b = gauss_elim(C,b)

L0 = np.array([[1,0,0,0],[-1,1,0,0],[-4,0,1,0],[-5,0,0,1]])
L1 = np.array([[1,0,0,0],[0,1,0,0],[0,-7,1,0],[0,-8,0,1]])
L2 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,-2,1]])
L0_ = linalg.inv(L0)
L1_ = linalg.inv(L1)
L2_ = linalg.inv(L2)

print 'L0:'
print L0
print 'L1:'
print L1
print 'L2:'
print L2
print '\n'
print '(L0_.(L1_.(L2_.U))) = '
print np.dot(L0_,np.dot(L1_,np.dot(L2_,U2)))
print '\n'
print 'L0_.L1_.L2_ = L:'
print np.dot(L0_,np.dot(L1_,L2_))
print '\n'


L0:
[[ 1  0  0  0]
 [-1  1  0  0]
 [-4  0  1  0]
 [-5  0  0  1]]
L1:
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0 -7  1  0]
 [ 0 -8  0  1]]
L2:
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0 -2  1]]


(L0_.(L1_.(L2_.U))) = 
[[  5.   7.   5.   9.]
 [  5.  14.   7.  10.]
 [ 20.  77.  41.  48.]
 [ 25.  91.  55.  67.]]


L0_.L1_.L2_ = L:
[[ 1.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 4.  7.  1.  0.]
 [ 5.  8.  2.  1.]]




## LU decomposition - implementation

So we can build an LU code easily from our Gaussian elimination code.  The final $U$ matrix we need here is as is already constructed through Gaussian elimination, the entries of $L$ we need are simply the ${A_{ik}}/{A_{kk}}$ multipliers we computed as part of the elimination, but threw away previously.

For a given pivot row $k$, for each of these multipliers (for every row below the pivot), as we compute them we know that we are going to transform the augmented matrix in order to achieve a new zero below the diagonal - we can store each multiplier in this position before moving on to the following row, we implicitly know that the diagonals of $L$ will be unity and so don't need to store these (and noting that we don't actually have a space for them anyway!). We then move on to the following pivot row, replacing the zeros in the corresponding column we are zero'ing, but again using the now spare space to store the multipliers.

For example, for the case above 

$$ A = 
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{black}5} & {\color{black}14} & {\color{black}7} & {\color{black}10}\\
    {\color{black}20} & {\color{black}77} & {\color{black}41} & {\color{black}48}\\
    {\color{black}25} & {\color{black}91} & {\color{black}55} & {\color{black}67}\\   
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}{1}} & {\color{black}{7}} & {\color{black}{2}} & {\color{black}{1}}\\
    {\color{blue}{4}} & {\color{black}{49}} & {\color{black}{21}} & {\color{black}{12}}\\
    {\color{blue}{5}} & {\color{black}{56}} & {\color{black}{30}} & {\color{black}{22}}\\    
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}1} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{blue}4} & {\color{blue}{7}} & {\color{black}{7}} & {\color{black}{5}}\\
    {\color{blue}5} & {\color{blue}{8}} & {\color{black}{14}} & {\color{black}{14}}\\
  \end{bmatrix}\quad\rightarrow\quad
  \begin{bmatrix}
    {\color{black}5} & {\color{black}7} & {\color{black}5} & {\color{black}9}\\
    {\color{blue}1} & {\color{black}7} & {\color{black}2} & {\color{black}1}\\
    {\color{blue}4} & {\color{blue}7} & {\color{black}7} & {\color{black}5}\\
    {\color{blue}5} & {\color{blue}8} & {\color{blue}{2}} & {\color{black}{4}}\\
  \end{bmatrix}
  = [\color{blue}L\backslash U]
$$



In [3]:
import numpy
from scipy import linalg
A=numpy.array([[ 5., 7.,   5.,  9.],
               [ 5., 14.,  7., 10.],
               [20., 77., 41., 48.],
               [25., 91. ,55., 67.]])
print A

[[  5.   7.   5.   9.]
 [  5.  14.   7.  10.]
 [ 20.  77.  41.  48.]
 [ 25.  91.  55.  67.]]


In [4]:
P,L,U=linalg.lu(A)

# P here is a 'permutation matrix' that performs swaps based upon partial pivoting
print P  

[[ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]]


In [5]:
# the lower-triangular matrix
print L  

[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.8        -0.375       1.          0.        ]
 [ 0.2         0.375       0.33333333  1.        ]]


In [6]:
# the upper-triangular matrix
print U  

[[ 25.          91.          55.          67.        ]
 [  0.         -11.2         -6.          -4.4       ]
 [  0.           0.          -5.25        -7.25      ]
 [  0.           0.           0.           0.66666667]]


In [7]:
# double check that P*L*U does indeed equal A
print numpy.dot(P, numpy.dot(L, U)) 

[[  5.   7.   5.   9.]
 [  5.  14.   7.  10.]
 [ 20.  77.  41.  48.]
 [ 25.  91.  55.  67.]]


Looking at the form of $P$ above, we can re-order the rows in advance and consider the LU decomposition of the matrix where $P=I$, as below. As we haven't bothered implementing pivoting ourselves, check that your LU implementation recreates the $A$, $L$ and $U$ below.

In [8]:
import numpy
from scipy import linalg
A=numpy.array([[25. ,91. ,55. ,67.],
               [ 5.,  7.,  5.,  9.], 
               [20., 77., 41., 48.],
               [ 5., 14., 7.,  10.]])
print A

[[ 25.  91.  55.  67.]
 [  5.   7.   5.   9.]
 [ 20.  77.  41.  48.]
 [  5.  14.   7.  10.]]


In [9]:
P,L,U=linalg.lu(A)
# P now should be the identity as pivoting no longer actually actions any row swaps with this A
print P  

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [10]:
print L

[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.8        -0.375       1.          0.        ]
 [ 0.2         0.375       0.33333333  1.        ]]


In [11]:
print U

[[ 25.          91.          55.          67.        ]
 [  0.         -11.2         -6.          -4.4       ]
 [  0.           0.          -5.25        -7.25      ]
 [  0.           0.           0.           0.66666667]]


In [12]:
print numpy.dot(P, numpy.dot(L, U))

[[ 25.  91.  55.  67.]
 [  5.   7.   5.   9.]
 [ 20.  77.  41.  48.]
 [  5.  14.   7.  10.]]


### <span style="color:blue">Exercise 3.2: LU decomposition</span>

Starting from your Gaussian elimination code produce a new code to compute the LU decomposition of a matrix.

## Partial pivoting

At the end of last week we commented that a problem could occur where the $A_{kk}$ we divide through by in the Gaussian elimination and/or back substitution algorithms might be (near) zero.

Using Gaussian elinination as an example, let's again consider the algorithm mid-way working on an arbitrary matrix system, i.e. assume that the first $k$ rows have already been transformed into upper-triangular form, while the equations/rows below are not yet in this form:

$$
\left[
  \begin{array}{rrrrrrr|r}
    A_{11} & A_{12} & A_{13} & \cdots & A_{1k}  & \cdots & A_{1n} & b_1 \\
    0      & A_{22} & A_{23} & \cdots & A_{2k} & \cdots & A_{2n} & b_2 \\
    0      & 0      & A_{33} & \cdots & A_{3k}  & \cdots & A_{3n} & b_3 \\
    \vdots & \vdots & \vdots & \ddots & \vdots  & \ddots & \vdots & \vdots \\
\hdashline    
    0      & 0      & 0      & \cdots & A_{kk}  & \cdots & A_{kn} & b_k \\    
    \vdots & \vdots & \vdots & \ddots & \vdots  & \ddots & \vdots & \vdots \\
    0      & 0      & 0      & \cdots & A_{nk}  & \cdots & A_{nn} & b_n \\
\end{array}
\right]
$$

Note we have drawn the horizontal dashed line one row higher, as we are not going to blindly asssume that it is wise to take the current row $k$ as the pivot row, and $A_{kk}$ as the so-called pivot element.

*Partial pivoting* selects the best pivot (row or element) as the one where the $A_{ik}$ (for $i\ge k$) value is largest (relative to the other values of components in its own row $i$), and then swaps this row with the current $k$ row.

To generalise our codes above we would simply need to search for this row, and perform the row swap operation.

# Challenge of the day

### <span style="color:blue">Exercise 3.3: Partial pivoting</span>

Implement partial pivoting.